In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
cd /content/drive/'My Drive'/'Colab Notebooks'/CS559_HW

/content/drive/My Drive/Colab Notebooks/CS559_HW


In [0]:
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import sys


images_train = np.load('./data/images_train.npy')
labels_train = np.load('./data/labels_train.npy')
images_validation = np.load('./data/images_validation.npy')
labels_validation = np.load('./data/labels_validation.npy')
images_test = np.load('./data/images_test.npy')
labels_test = np.load('./data/labels_test.npy')

In [0]:
def model(learning_rate = 0.001, dr_keep=0.5, l2_scale=0.1, filter_coef=1):
  x = tf.placeholder(tf.float32, [None, 91*91], name="features_placeholder")
  x_shaped = tf.reshape(x, [-1, 91, 91, 1])
  y = tf.placeholder(tf.int32, [None, 1], name="labels_placeholder")
  train_mode = tf.placeholder(tf.bool, name="train_mode_placeholder")

  
  conv1 = tf.contrib.layers.conv2d(
                                   inputs=x_shaped,
                                   num_outputs=24*filter_coef,
                                   kernel_size=5,
                                   stride=2,
                                   padding='VALID',
                                   # activation_fn=tf.nn.relu,
                                   # weights_initializer=w_initializer,
                                   weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_scale),
                                   # biases_initializer=tf.zeros_initializer(),
                                   # name = "Conv2d_1"
                                  )
  
  bn1 = tf.layers.batch_normalization(
                                      inputs=conv1,
                                      training=train_mode,
                                      momentum=0.9,
                                      # name = "BatchNorm_1"
                                     )
  
  conv2 = tf.contrib.layers.conv2d(
                                   inputs=bn1,
                                   num_outputs=36*filter_coef,
                                   kernel_size=5,
                                   stride=2,
                                   padding='VALID',
                                   # activation_fn=tf.nn.relu,
                                   # weights_initializer=w_initializer,
                                   weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_scale),
                                   # biases_initializer=tf.zeros_initializer(),
                                   # name = "Conv2d_2"
                                  )
  
  conv3 = tf.contrib.layers.conv2d(
                                   inputs=conv2,
                                   num_outputs=48*filter_coef,
                                   kernel_size=5,
                                   stride=2,
                                   padding='VALID',
                                   # activation_fn=tf.nn.relu,
                                   # weights_initializer=w_initializer,
                                   weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_scale),
                                   # biases_initializer=tf.zeros_initializer(),
                                   # name = "Conv2d_3"
                                  )
  
  bn2 = tf.layers.batch_normalization(
                                      inputs=conv3,
                                      training=train_mode,
                                      momentum=0.9,
                                      # name = "BatchNorm_2"
                                     )

  conv4 = tf.contrib.layers.conv2d(
                                   inputs=bn2,
                                   num_outputs=64*filter_coef,
                                   kernel_size=3,
                                   stride=1,
                                   padding='VALID',
                                   # activation_fn=tf.nn.relu,
                                   # weights_initializer=w_initializer,
                                   weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_scale),
                                   # biases_initializer=tf.zeros_initializer(),
                                   # name = "Conv2d_4"
                                  )
  
  conv5 = tf.contrib.layers.conv2d(
                                   inputs=conv4,
                                   num_outputs=64*filter_coef,
                                   kernel_size=3,
                                   stride=1,
                                   padding='VALID',
                                   # activation_fn=tf.nn.relu,
                                   # weights_initializer=w_initializer,
                                   weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_scale),
                                   # biases_initializer=tf.zeros_initializer(),
                                   # name = "Conv2d_5"
                                  )
  
    
  bn3 = tf.layers.batch_normalization(
                                      inputs=conv5,
                                      training=train_mode,
                                      momentum=0.9,
                                      # name = "BatchNorm_3"
                                     )
                                     
  flattened = tf.contrib.layers.flatten(
                                        inputs=bn3,
                                        # name="Flatten"
                                       )

  
  dr1 = tf.contrib.layers.dropout(
                                  inputs=flattened,
                                  keep_prob=dr_keep,
                                  is_training=train_mode,
                                  # name="Dropout_1"
                                 )
  
  fc1 = tf.contrib.layers.fully_connected(
                                          inputs=dr1,
                                          num_outputs=1164,
                                          # activation_fn=tf.nn.relu,
                                          # weights_initializer=w_initializer,
                                          weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_scale),
                                          # biases_initializer=tf.zeros_initializer(),
                                          # name="FullyConnected_1"
                                         )
                                     
  bn4 = tf.layers.batch_normalization(
                                      inputs=fc1,
                                      training=train_mode,
                                      momentum=0.9,
                                      # name="BatchNorm_4"
                                     )
      
  dr2 = tf.contrib.layers.dropout(
                                  inputs=bn4,
                                  keep_prob=dr_keep,
                                  is_training=train_mode,
                                  # name="Dropout_2"
                                 )
  
  fc2 = tf.contrib.layers.fully_connected(
                                          inputs=dr2,
                                          num_outputs=100,
                                          # activation_fn=tf.nn.relu,
                                          # weights_initializer=w_initializer,
                                          weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_scale),
                                          # biases_initializer=tf.zeros_initializer(),
                                          # name="FullyConnected_2"
                                         )
                                     
    
  bn5 = tf.layers.batch_normalization(fc2,
                                      training=train_mode,
                                      momentum=0.9,
                                      # name="BatchNorm_5"
                                     )
      
  dr3 = tf.contrib.layers.dropout(
                                  inputs=bn5,
                                  keep_prob=dr_keep,
                                  is_training=train_mode,
                                  # name="Dropout_3"
                                 )
  
  fc3 = tf.contrib.layers.fully_connected(
                                          inputs=dr3,
                                          num_outputs=50,
                                          # activation_fn=tf.nn.relu,
                                          # weights_initializer=w_initializer,
                                          weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_scale),
                                          # biases_initializer=tf.zeros_initializer(),
                                          # name="FullyConnected_3"
                                         )
                                     
    
  bn6 = tf.layers.batch_normalization(fc3,
                                      training=train_mode,
                                      momentum=0.9,
                                      # name="BatchNorm_6"
                                     )
  
  dr4 = tf.contrib.layers.dropout(
                                  inputs=bn6,
                                  keep_prob=dr_keep,
                                  is_training=train_mode,
                                  # name="Dropout_4"
                                 )
  
  fc4 = tf.contrib.layers.fully_connected(
                                          inputs=dr4,
                                          num_outputs=10,
                                          # activation_fn=tf.nn.relu,
                                          # weights_initializer=w_initializer,
                                          weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_scale),
                                          # biases_initializer=tf.zeros_initializer(),
                                          # name="FullyConnected_4"
                                         )
    
  bn7 = tf.layers.batch_normalization(
                                      inputs=fc4,
                                      training=train_mode,
                                      momentum=0.9,
                                      # name="BatchNorm_7"
                                     )
      
  dr5 = tf.contrib.layers.dropout(
                                  inputs=bn7,
                                  keep_prob=dr_keep,
                                  is_training=train_mode,
                                  # name="Dropout_5"
                                 )
  
  fc5 = tf.contrib.layers.fully_connected(
                                          inputs=dr5,
                                          num_outputs=1,
                                          # activation_fn=tf.nn.relu,
                                          # weights_initializer=w_initializer,
                                          weights_regularizer=tf.contrib.layers.l2_regularizer(scale=l2_scale),
                                          # biases_initializer=tf.zeros_initializer(),
                                          # name="FullyConnected_5"
                                         )
  # For real output round and clip
  output = tf.round(fc5)
  output = tf.cast(output, tf.int32)
  output = tf.clip_by_value(output, 1, 80)
  
  # MAE for evaluating model 
  loss_mae_round = tf.losses.absolute_difference(y, output)  
  
  # Loss function for optimizer
  loss_mae = tf.losses.mean_squared_error(y, fc5)
  
  # L2 Regularization
  reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
  loss_mae += sum(reg_losses)

  # Adam Optimizer
  optimizer = tf.contrib.optimizer_v2.AdamOptimizer(learning_rate=learning_rate)
  
  # For BatchNorm, the moving_mean and moving_variance need to be updated while training
  update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
  
  # Optimize and group
  train_op = optimizer.minimize(loss_mae)  
  train_op = tf.group([train_op, update_ops])
  
  return x, y, output, train_mode, train_op, loss_mae_round

In [0]:
def restore_model(model_name="model", learning_rate= 0.0001, dr_keep=0.6, l2_scale=0.01, filter_coef=2):
  tf.reset_default_graph()
  x, y, output, train_mode, train_op, loss_mae_round = model(learning_rate=learning_rate, dr_keep=dr_keep, l2_scale=l2_scale, filter_coef=filter_coef)

  log_directory = "./logs/" + model_name + "/"
  
  saver = tf.train.Saver()

  with tf.Session() as sess:
    saver.restore(sess, log_directory + model_name + ".ckpt")
    print(model_name, "restored.")

    validation_loss = sess.run(loss_mae_round, feed_dict={x: images_validation, y: labels_validation, train_mode:False})
    test_loss = sess.run(loss_mae_round, feed_dict={x: images_test, y: labels_test, train_mode:False})
    
  return validation_loss, test_loss

In [0]:
def run_model(learning_rate = 0.001, epochs = 100, batch_size = 50, dr_keep=0.5, l2_scale=0.1, filter_coef=1, early_stop = 10, save_result=True, outfile_name=None):
  global images_train
  global images_validation
  global images_test
  
  if not outfile_name:
    outfile_name = "model_lr" + str(learning_rate) + "_bs" + str(batch_size) + "_drk" + str(dr_keep) + "_l2" + str(l2_scale) + "_fc" + str(filter_coef)
  print("\nStarting " + outfile_name)
  log_directory = "./logs/" + outfile_name + "/"
  
  if not os.path.exists(log_directory):
    os.makedirs(log_directory)
  
  tf.reset_default_graph()
  x, y, output, train_mode, train_op, loss_mae_round = model(learning_rate, dr_keep, l2_scale, filter_coef)
  
  train_total_batch = int(len(labels_train) / batch_size)

  # setup the initialisation operator
  init_op = tf.global_variables_initializer()
  saver = None
  if save_result:
    saver = tf.train.Saver()
    
  with tf.Session() as sess:
    """ # Image standardization
    with sess.as_default():
      images_train = tf.image.per_image_standardization(images_train.reshape([-1, 91,91,1])).eval()
      images_validation = tf.image.per_image_standardization(images_validation.reshape([-1, 91,91,1])).eval()
      images_test = tf.image.per_image_standardization(images_test.reshape([-1, 91,91,1])).eval()
    """
    # initialize the variables
    sess.run(init_op)
    
    train_losses = []
    validation_losses = []
    
    best_validation_loss = np.inf
    best_validation_loss_epoch = 0

    early_stop_counter = 0
    
    for epoch in range(epochs):
      sum_loss = 0
      for i in range(train_total_batch):
        batch_x = images_train[i*batch_size:i*batch_size+batch_size]
        batch_y = labels_train[i*batch_size:i*batch_size+batch_size]
        _, c= sess.run([train_op, loss_mae_round], feed_dict={x: batch_x, y: batch_y, train_mode:True})
        sum_loss += c

      train_avg_loss = sum_loss / train_total_batch

      # Validation loss after the epoch
      validation_loss = sess.run(loss_mae_round, feed_dict={x: images_validation, y: labels_validation, train_mode:False})

      train_losses.append(train_avg_loss)
      validation_losses.append(validation_loss)

      print("Epoch:", (epoch + 1), ", train loss:", "{:.3f}".format(train_avg_loss), ", validation loss: {:.3f}".format(validation_loss))

      if validation_loss < best_validation_loss:
        early_stop_counter = 0
        best_validation_loss = validation_loss
        best_validation_loss_epoch = epoch + 1
        if save_result:
          save_path = saver.save(sess, log_directory + outfile_name + ".ckpt")
          print("Model saved in path: %s" % save_path)
      else:
        early_stop_counter += 1

      if early_stop_counter >= early_stop:
        print("Early stopping is trigger at step: {} loss:{}".format(epoch + 1, best_validation_loss))
        break
    if not save_result:
      test_loss = sess.run(loss_mae_round, feed_dict={x: images_test, y: labels_test, train_mode:False})
      print("\nTraining complete! Test loss: {:.3f}".format(test_loss))

  if save_result:
    validation_loss, test_loss = restore_model(outfile_name, learning_rate=learning_rate, dr_keep=dr_keep, l2_scale=l2_scale, filter_coef=filter_coef)

    with open("./results/" + outfile_name + '.txt', 'w') as f:
      f.write("Epoch Train_Loss Validation_Loss\n")
      for i in range(len(train_losses)):
        f.write("{0} {1:.3f} {2:.3f}\n".format(i + 1, train_losses[i], validation_losses[i]))
      f.write("Training complete! Best validation at epoch {}. Validation loss: {:.3f}, Test loss: {:.3f}".format(best_validation_loss_epoch, validation_loss, test_loss))

    print("\nTraining complete! Best validation at epoch {}. Validation loss: {:.3f}, Test loss: {:.3f}".format(best_validation_loss_epoch, validation_loss, test_loss))

In [0]:
def graph_tensorboard():   
  tf.reset_default_graph()
  x, y, output, train_mode, train_op, loss_mae_round = model(learning_rate=0.01, dr_keep=0.6, l2_scale=0.1, filter_coef=2)

  init_op = tf.global_variables_initializer()

  with tf.Session() as sess:
      sess.run(init_op)
      writer = tf.summary.FileWriter("output", sess.graph)
      print(sess.run([train_op, loss_mae_round], feed_dict={x: images_test, y: labels_test, train_mode:True}))
      writer.close()

In [48]:
run_model(learning_rate=0.0001, epochs=3000, batch_size=200, dr_keep=0.6, l2_scale=0.01, filter_coef=2, early_stop=500, save_result=True, outfile_name="LastVersion")


Starting LastVersion
Epoch: 1 , train loss: 30.735 , validation loss: 30.078
Model saved in path: ./logs/LastVersion/LastVersion.ckpt
Epoch: 2 , train loss: 30.629 , validation loss: 29.753
Model saved in path: ./logs/LastVersion/LastVersion.ckpt
Epoch: 3 , train loss: 30.517 , validation loss: 30.297
Epoch: 4 , train loss: 30.394 , validation loss: 30.238
Epoch: 5 , train loss: 30.296 , validation loss: 30.214
Epoch: 6 , train loss: 30.163 , validation loss: 30.074
Epoch: 7 , train loss: 30.086 , validation loss: 29.991
Epoch: 8 , train loss: 30.000 , validation loss: 29.928
Epoch: 9 , train loss: 29.929 , validation loss: 29.870
Epoch: 10 , train loss: 29.885 , validation loss: 29.841
Epoch: 11 , train loss: 29.800 , validation loss: 29.739
Model saved in path: ./logs/LastVersion/LastVersion.ckpt
Epoch: 12 , train loss: 29.747 , validation loss: 29.727
Model saved in path: ./logs/LastVersion/LastVersion.ckpt
Epoch: 13 , train loss: 29.684 , validation loss: 29.694
Model saved in pat

In [49]:
 restore_model(model_name="LastVersion")

INFO:tensorflow:Restoring parameters from ./logs/LastVersion/LastVersion.ckpt
LastVersion restored.


(6.4185743, 6.4857635)